In [1]:
import time
import torch
from transformers import GPTJForCausalLM, GPT2Tokenizer
import torch

/home/azureuser/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Will need at least 13-14GB of Vram for CUDA
if torch.cuda.is_available():
    model =  GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B",cache_dir="/datadrive/cache_dir/", torch_dtype=torch.float16).cuda()
else:
    print("GPU not available, check driver and cuda installation and reboot if neccessary")
    #model =  GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", torch_dtype=torch.float16)

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [4]:
environment_description = """
Name: Max  Occupation: Consultant Salary: 500
Name: Sam  Occupation: Consultant Salary: 600
Name: James Occupation: Consultant Salary: 700"""

In [5]:
from flask import Flask, request, jsonify
import json

app = Flask(__name__)

@app.route('/generate', methods=['GET']) 
def generate():

    data = json.loads(request.json, strict=False)
    print(data)

    environment_description = data["body"]

    input_text = environment_description
    input_ids = tokenizer.encode(str(input_text), return_tensors='pt').cuda()

    output = model.generate(
        input_ids,
        do_sample=True,
        top_p=0.99,
        top_k=10,
        temperature=0.9,
        max_new_tokens=int(data['token'])*10,
        penalty_alpha=0.6
        )

    out = tokenizer.decode(output[0], skip_special_tokens=True)
    
    
    result = {"result":out}
    
    return jsonify(result)

app.run(port='5002')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002 (Press CTRL+C to quit)
[2022-11-11 07:30:51,835] ERROR in app: Exception on /generate [GET]
Traceback (most recent call last):
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/tmp/ipykernel_2228/2231741901.py", line 10, in generate
    data = json.loads(request.json, strict=False)
  File "/home/azureuser/anaconda3/envs/ldm/lib/python3.8/json/__init__.py", line 370, in loads
    retu